In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torchvision.datasets import CocoDetection

# Backbone Init

In [6]:
class ResNetBackBone(nn.Module):
    def __init__(self, pretrained_model):
        super(ResNetBackBone, self).__init__()
        self.features = nn.Sequential(
            *list(pretrained_model.children())[:-2]
        )

        self.avgpool = nn.AvgPool2d(kernel_size=1, stride=1, padding=0)
        self.fc = nn.Conv2d(512, 256, 1)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.fc(x)
        return x

In [7]:
resnet18 = ResNetBackBone(pretrained_model=torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True))

Using cache found in /Users/bartoszgawin/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
for param in resnet18.features.parameters():
    param.requires_grad = False

resnet18.avgpool.requires_grad = False
resnet18.fc.requires_grad = True

# Building the Transformer blocks

In [9]:
class EncoderBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, dropout = 0):
        super(EncoderBlock, self).__init__()
        self.attention_matrices = None

        self.multi_head_attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)
        self.norm_1 = nn.LayerNorm(embedding_dim)
        self.dropout_1 = nn.Dropout(p = dropout)

        self.fully_connected = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=embedding_dim, kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=embedding_dim, out_channels=embedding_dim, kernel_size=1, stride=1, padding=0)
        )

        self.dropout_2 = nn.Dropout(p = dropout)
        self.norm_2 = nn.LayerNorm(embedding_dim)

    def forward(self, X, positional_encoding):
        spatially_encoded = X + positional_encoding
        attention_output, self.attention_matrices = self.multi_head_attention(X, spatially_encoded, spatially_encoded, average_attn_weights=True)

        print(f'attention_output dim: {attention_output.shape}')

        attention_output = self.norm_1(X + self.dropout_1(attention_output))
        fc_output = self.fully_connected(attention_output.permute(0, 2, 1)).permute(0, 2, 1)
        output = self.norm_2(attention_output + self.dropout_2(fc_output))
        return output.permute(0, 2, 1)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, dropout = 0):
        self.attention_matrices_1 = None
        self.attention_matrices_2 = None

        self.multi_head_attention_1 = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)
        self.norm_1 = nn.LayerNorm(embedding_dim)
        self.dropout_1 = nn.Dropout(p = dropout)


        self.multi_head_attention_2 = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)
        self.norm_2 = nn.LayerNorm(embedding_dim)
        self.dropout_2 = nn.Dropout(p = dropout)

        self.fully_connected = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=embedding_dim, kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=embedding_dim, out_channels=embedding_dim, kernel_size=1, stride=1, padding=0)
        )

        self.norm_3 = nn.LayerNorm(embedding_dim)
        self.dropout_3 = nn.Dropout(p = dropout)



# Checking the Code

In [10]:
batch_size = 10
patch_size = 4
height_and_width = 28
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
dimensions = int(D / H)

multi_head_attention = nn.MultiheadAttention(D, H, batch_first=True)
input = torch.randn(batch_size, N + 1, D)
output = multi_head_attention(input, input, input, average_attn_weights=True)
output

(tensor([[[-6.2313e-02,  5.3883e-02, -4.7997e-02,  ...,  5.3505e-02,
            3.1756e-03, -5.7489e-02],
          [-2.2390e-02,  9.8850e-02,  5.2497e-03,  ...,  1.8847e-02,
           -2.1615e-02, -3.9953e-02],
          [-8.9739e-02,  3.3419e-02,  4.3425e-03,  ...,  3.9003e-02,
           -7.0934e-03, -9.7791e-02],
          ...,
          [-7.5032e-02,  7.7434e-02,  4.5893e-02,  ..., -9.8724e-03,
           -4.8991e-02, -7.6481e-02],
          [-4.2733e-02, -2.8502e-02, -8.1618e-02,  ...,  3.6784e-02,
           -2.3941e-02, -3.0400e-02],
          [-1.7001e-01,  1.5190e-01, -1.1051e-01,  ...,  8.5148e-02,
           -2.4537e-02, -3.2341e-04]],
 
         [[ 4.6253e-02,  5.3670e-03,  1.2932e-01,  ..., -8.6938e-02,
           -1.7344e-02, -3.5458e-02],
          [-4.3330e-02,  4.7686e-02,  1.1634e-01,  ..., -1.0411e-01,
           -1.4437e-02,  1.2916e-02],
          [-2.3927e-02,  5.7526e-02,  1.2591e-01,  ..., -6.1286e-02,
           -1.3235e-02,  5.2719e-03],
          ...,
    

In [11]:
input.permute(0, 2, 1).permute(0, 2, 1).shape

torch.Size([10, 50, 64])

In [17]:
batch_size = 10
patch_size = 4
height_and_width = 28
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
C = 64
dimensions = int(D / H)

multi_head_attention = EncoderBlock(D, H)
input = torch.randn(batch_size, N+1, D)
print(input.shape)
output = multi_head_attention(input, input)

torch.Size([10, 50, 64])
attention_output dim: torch.Size([10, 50, 64])


In [13]:
from torchvision import transforms
from medmnist import DermaMNIST
from torch.utils.data import DataLoader

train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

train_download = DermaMNIST(split='train', transform=train_transformations, download=True)

Using downloaded and verified file: /Users/bartoszgawin/.medmnist/dermamnist.npz


In [14]:
x = torch.rand(2, 3, 224, 224)
print(x.shape)
output = resnet18(x)

torch.Size([2, 3, 224, 224])


In [15]:
output.shape

torch.Size([2, 256, 7, 7])

# Source Code of ResNet

In [16]:
# Source code of ResNet
class ResNet(nn.Module):
    def __init__(
            self,
            block: Type[Union[BasicBlock, Bottleneck]],
            layers: List[int],
            num_classes: int = 1000,
            zero_init_residual: bool = False,
            groups: int = 1,
            width_per_group: int = 64,
            replace_stride_with_dilation: Optional[List[bool]] = None,
            norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
            self,
            block: Type[Union[BasicBlock, Bottleneck]],
            planes: int,
            blocks: int,
            stride: int = 1,
            dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

NameError: name 'Type' is not defined